### Assisgment:
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.


In [2]:
import os 
from pathlib import Path 
from dotenv import load_dotenv
current_script_dir = Path().parent
dotenv_file_path = Path.cwd().parent  / '.env'
dotenv_file_path = dotenv_file_path.resolve()
load_dotenv(dotenv_path=dotenv_file_path)

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [43]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,PydanticOutputParser
from pydantic import BaseModel,Field
from langchain_openai import ChatOpenAI
from typing import List

In [47]:
class Productdetails(BaseModel):
    "This class contains the details information about the product like product name, price in USD and product Description"
    productname: str =Field(default="unknown",description="Name of the product")
    productdescription :str = Field(default="unknown",description="Details of the product")
    price_USD:int = Field(description="price in USD",default=0)
   


In [49]:
Productdetails(**{"productname":"dfhg","productdescription":"skgvb","price_USD":789})

Productdetails(productname='dfhg', productdescription='skgvb', price_USD=789)

In [21]:
prompt = ChatPromptTemplate([
    ("system","Your are a simple AI Assitant.Provide all your outputs in json format. \
    Your job is to provide the product name , product details and price(you can give an estimated price based on your knowldge) based on the user input.If you dont know about a particular product please say unknown "),
    ("user","{input}")
])

In [22]:
model= ChatGroq(model="gemma2-9b-it")

In [ ]:
output_parser = JsonOutputParser(pydantic_object=Productdetails)

In [26]:
chain = prompt | model |output_parser

In [27]:
chain.invoke({"input":"what you think about Iphone 13 pro"})

{'product_name': 'iPhone 13 Pro',
 'product_details': 'A premium smartphone with a 6.1-inch Super Retina XDR display, a triple-camera system featuring a telephoto lens, A15 Bionic chip, and up to 1TB of storage.',
 'price': 999}

In [28]:
chain.invoke({"input":"Hi"})

{'status': 'ready',
 'message': "Hi! I'm ready to help. What product are you interested in?"}

In [29]:
chain.invoke({"input":"what you think about samrt ring"})

{'product_name': 'Smart Ring',
 'product_details': 'A smart ring is a wearable electronic device that is worn on the finger. It can track fitness data, control smart devices, and provide other features like contactless payments and notifications.',
 'price': 'Estimated price: $100 - $500'}

In [32]:
# other methods

In [59]:
class Products(BaseModel):
    'Contains List of all the products'
    products:List[Productdetails] = Field(description="A list of product information objects.")
parser = PydanticOutputParser(pydantic_object=Products)
prompt2 = PromptTemplate(
    template="""You are a highly skilled product information extractor.
Your task is to identify and extract product name from the user's query.And respond with product name,product description and price.
{format_instructions}
If multiple products are mentioned, extract information for all of them.
If a price is not exact, provide a approximate value.

User Query: {query}
""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
openai_model = ChatOpenAI(model="o1-mini")
chain2 = prompt2|openai_model|parser

In [60]:
response2 = chain2.invoke({"query":"what you think about iphone and samsung"})

In [62]:
for i in response2.products:
    print(i.productname,"\n")
    print(i.productdescription,"\n")
    print(i.price_USD,"\n")

iPhone 15 

Apple's latest smartphone featuring improved cameras, enhanced performance with the new A17 chip, and longer battery life. 

999 

Samsung Galaxy S23 

Samsung's flagship smartphone with a dynamic AMOLED display, powerful performance, and versatile camera system. 

799 

